In [ ]:
!git clone https://github.com/HoangVuSnape/LLM_MiAI.git

Cloning into 'LLM_MiAI'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 52 (delta 11), reused 48 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 5.67 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
%cd /content/LLM_MiAI/StableDiffusionMiIAi

/content/LLM_MiAI/StableDiffusionMiIAi


In [ ]:
# !pip install -q -r setup.txt

In [ ]:
!pip install -q flask torch diffusers transformers accelerate

In [ ]:
!pip -q install pyngrok

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# Định nghĩa tham số
rand_seed = torch.manual_seed(42)
NUM_INFERENCE_STEPS = 50
GUIDANCE_SCALE = 0.75
HEIGHT = 512
WIDTH = 512

# Danh sách model
model_list = ["nota-ai/bk-sdm-small",
              "CompVis/stable-diffusion-v1-4",
              "runwayml/stable-diffusion-v1-5",
              "prompthero/openjourney",
              "hakurei/waifu-diffusion",
              "stabilityai/stable-diffusion-2-1",
              "dreamlike-art/dreamlike-photoreal-2.0"
              ]


def create_pipeline(model_name = model_list[5]):
    # Nếu máy có GPU CUDA
    if torch.cuda.is_available():
        print("Using GPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype = torch.float16,
            use_safetensors = True
        ).to("cuda")
    elif torch.backends.mps.is_available():
        print("Using MPS")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("mps")
    else:
        print("Using CPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float32,
            use_safetensors=True
        )
    return pipeline

def text2img(prompt, pipeline):
    images = pipeline(
        prompt,
        guidance_scale = GUIDANCE_SCALE,
        num_inference_steps = NUM_INFERENCE_STEPS,
        generator = rand_seed,
        num_images_per_request = 1,
        height = HEIGHT,
        width = WIDTH
    ).images

    return images[0]

In [ ]:
from flask import Flask, request, render_template
# from text2img_model import create_pipeline, text2img
from pyngrok import ngrok, conf

from google.colab import userdata
import os

# Khởi tạo Flask app
app =  Flask(__name__)

# Định nghĩa tham số
IMAGE_PATH  = "static/output.jpg"

# Tạo thư mục 'static' nếu chưa tồn tại
if not os.path.exists("static"):
    os.makedirs("static")

conf.get_default().auth_token = userdata.get('ngrok_token')

public_url = ngrok.connect(8888).public_url

print("My tunel = ", public_url)
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


# Khởi tạo pipeline
pipeline = create_pipeline()

@app.route("/", methods = ['GET', 'POST'])
def index():
    # Kiểm tra xem có phải người dùng view trang web không
    if request.method == "GET":
        # Trả về giao diện trang web
        return render_template("index.html")
    else:
        # return 1
        # Xử lý việc người dùng submit prompt-> sinh ảnh -> trả về
        user_input = request.form["prompt"]

        print("Start gen....")
        im = text2img(user_input, pipeline)
        print("Finish gen....")
        im.save(IMAGE_PATH)

        return render_template("index.html", image_url = IMAGE_PATH)

if __name__ =='__main__':
    app.run(debug=False, host='0.0.0.0', port=8888, use_reloader=False)

My tunel =  https://68f7-35-203-153-153.ngrok-free.app
Using GPU


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8888
 * Running on http://172.28.0.12:8888
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 07:58:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 07:58:18] "GET /favicon.ico HTTP/1.1" 404 -


Start gen....


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 07:58:38] "POST / HTTP/1.1" 200 -


Finish gen....


INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 07:58:38] "GET /static/output.jpg HTTP/1.1" 200 -
